Import necessary libs:

In [1]:
import pandas as pd
import os
import glob

Define variables to hold json paths:

In [2]:
chat_messages_path = './chatguru'

Get chat files:

In [3]:
chat_files = glob.glob(os.path.join(chat_messages_path,'*.txt'))
for chat in chat_files:
    print(chat.split('\\')[1].split('.')[0])

5ef4e31c39d1eb3622ef0bb3
5ef4e31d39d1eb3622ef0bbf
5ef4e31d39d1eb3622ef0bc4
5ef4e31d39d1eb3622ef0bd0
5ef4e31d39d1eb3622ef0bd8
5ef4e31e39d1eb3622ef0bde
5ef4e31e39d1eb3622ef0be0
5ef4e31e39d1eb3622ef0be6
5ef4e31e39d1eb3622ef0be9
5ef4e31e39d1eb3622ef0bf2
5ef4e31e39d1eb3622ef0bf5
5ef4e31e39d1eb3622ef0bfb
5ef9dbee95426316e6564527
5efa019639d1eb3622ef5f46
5efa29e3a4b9132668dbc534
5efa29fa33951ad3801a7f07
5efa30ccffd0dd3f13a136fb
5efa353daa6d294b22759043
5efa4b3b7e1eefe0e1d8360a
5efb1e826795cfd419fa4996
5efb30840080f7901e05f9f4
5efb309c23cdd5dfb587afc5
5efb600ca73a576ea04907b0
5efb6023a3f771d45b260325
5efb6ec8764c2f22479d306b
5efb6ee0277e6cd127e30f13
5efb6f3807ffcc790c722b6b
5efb6f5081e50b662c226f11
5efb6fb32f3d2fcf23619147
5efb6fcbdf887bb6e4c2894d
5efb7d7f23cdd5dfb587b07f
5efb8cd43adc06f06c4d0881
5efb8e25ada95594adda1b4e
5efb8ef639d1eb3622ef7d4d
5efb9aba2dd4a4180991607b
5efc811439d1eb3622ef8ee1
5efc83f25f4367ee83e5b2df
5efc8e1058479681c729fbd7
5efca404e86e5b24cce50e5e
5efca45c1715f0653be07f57


Get lines in text files:

In [35]:
# Create dictionaries to structure dataframe data
chats_dict = {'id':[],'num_client_messages':[],'num_service_messages':[]}
messages_dict = {'id':[],'chat_id':[],'order_in_chat':[],'text':[],'source':[]}

# Start global chat and message counter
message_id = 0
chat_id = 1
message_order = 0
num_client_messages = 0
num_service_messages = 0
# i = 0

# Iterate through .txt chat files
for chat in chat_files:
    with open(chat, 'r',encoding='utf-8') as file:
        # Iterate through each line in chat file
        for line in file:
            
#             if i == 0:
#                 print(line)
                
            # Skip empty line
            if line.strip() == '':
                continue
            
            # End of chat (counters: update chat and restart messages)
            if line.strip() == '_(Não é preciso responder essa mensagem.)_':
                chats_dict['id'].append(chat_id)
                chats_dict['num_client_messages'].append(num_client_messages)
                chats_dict['num_service_messages'].append(num_service_messages)
                chat_id += 1
                num_client_messages = 0
                num_service_messages = 0
                message_order = 0
                continue
                
            word_list = line.split(' ')

            # If first messages don't have a defined sender, skip them
            if message_id == 0 and (word_list[0] != 'Atendente:' and word_list[0] != 'Cliente:'):
                print(word_list[0])
                continue

            # Case sender is 'Atendente'
            if word_list[0] == 'Atendente:':
                # Atualiza e adiciona a id de mensagem
                message_id += 1
                messages_dict['id'].append(message_id)
                
                # Adiciona o chat correspondente
                messages_dict['chat_id'].append(chat_id)
                
                # Atualiza e adiciona a ordem de mensagem
                message_order += 1
                messages_dict['order_in_chat'].append(message_order)
                
                # Atualiza o número de mensagens de atendentes no chat
                num_service_messages += 1
                
                # Adiciona o texto da linha no dict de mensagens
                messages_dict['text'].append(line)
                
                # Adiciona a fonte da mensagem
                messages_dict['source'].append('A')
                

            # Case sender is 'Cliente'
            elif word_list[0] == 'Cliente:':
                # Atualiza a id de mensagem
                message_id += 1
                messages_dict['id'].append(message_id)
                
                
                # Adiciona o chat correspondente
                messages_dict['chat_id'].append(chat_id)
                
                # Atualiza a ordem de mensagem
                message_order += 1
                messages_dict['order_in_chat'].append(message_order)
                
                # Atualiza o número de mensagens de atendentes no chat
                num_client_messages += 1
                
                # Adiciona o texto da linha no dict de mensagens
                messages_dict['text'].append(line)
                
                # Adiciona a fonte da mensagem
                messages_dict['source'].append('C')

            # Caso seja a continuação de uma mensagem da linha anterior
            else:
                # Adiciona o texto da linha ao anterior
                messages_dict['text'][message_id-1] = messages_dict['text'][message_id-1] + line
        i += 1


Create chats dataframe:

In [36]:
chat_df = pd.DataFrame(data=chats_dict)
print('Info about chats dataframe:\n')
chat_df.info()

Info about chats dataframe:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5251 entries, 0 to 5250
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   id                    5251 non-null   int64
 1   num_client_messages   5251 non-null   int64
 2   num_service_messages  5251 non-null   int64
dtypes: int64(3)
memory usage: 123.2 KB


In [37]:
chat_df.head(10)

,id,num_client_messages,num_service_messages
0,1,5,9
1,2,13,11
2,3,6,8
3,4,10,8
4,5,2,5
5,6,6,6
6,7,8,113
7,8,28,1393
8,9,0,3
9,10,0,4


Perform same actions for messages dictionary:

In [38]:
message_df = pd.DataFrame(data=messages_dict)
print('Info about messages dataframe:\n')
message_df.info()

Info about messages dataframe:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154975 entries, 0 to 154974
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             154975 non-null  int64 
 1   chat_id        154975 non-null  int64 
 2   order_in_chat  154975 non-null  int64 
 3   text           154975 non-null  object
 4   source         154975 non-null  object
dtypes: int64(3), object(2)
memory usage: 5.9+ MB


In [41]:
message_df.tail(30)

,id,chat_id,order_in_chat,text,source
154945,154946,5251,5,"Atendente: *Arthur Ferragini:* \nOlá, Estefâni...",A
154946,154947,5251,6,Cliente: Sim\n,C
154947,154948,5251,7,Cliente: E-mail atual: biancaotmz@gmail.com\nN...,C
154948,154949,5251,8,Cliente: Eu gostaria de pedir para vocês passa...,C
154949,154950,5251,9,"Atendente: *Arthur Ferragini:* \nPronto, o nom...",A
154950,154951,5251,10,Cliente: ?\n,C
154951,154952,5251,11,Atendente: *Arthur Ferragini:* \nNesse caso é ...,A
154952,154953,5251,12,Cliente: Eu sou a responsável\n,C
154953,154954,5251,13,"Atendente: *Arthur Ferragini:* \nEstefânia, co...",A
154954,154955,5251,14,Cliente: Sim. Ela já entrou em contato com voc...,C


In [42]:
print('Number of client messages:')
print(f"{message_df[message_df['source'] == 'C'].shape[0]}")
print('\nPercentage of client messages:')
print(f"{round(100 * (message_df[message_df['source'] == 'C'].shape[0] / message_df.shape[0]),2)} %")

Number of client messages:
72773

Percentage of client messages:
46.96 %


In [43]:
chat_df['id'].unique().size

5251

In [108]:
sorted_messages_df = message_df.sort_values(by=['chat_id','send_date'])

In [109]:
sorted_messages_df.head(100)

,id,send_date,chat_id,status,text,quotes
396492,5ff6cbd5d9e6c058abe5416e,2021-01-07T08:52:37.647Z,5ff6cbd5d9e6c058abe54168,sent,[texto removido - regras GLPD],sem quotes
396495,5ff8c723820c4f6e1a53d3ba,2021-01-08T20:57:07.641Z,5ff6cbd5d9e6c058abe54168,sent,[texto removido - regras GLPD],sem quotes
396496,5ff98913357e03f135d256df,2021-01-09T10:44:35.305Z,5ff6cbd5d9e6c058abe54168,sent,[texto removido - regras GLPD],sem quotes
396497,5ff98c67ee1a9fc42d736c2f,2021-01-09T10:58:47.225Z,5ff6cbd5d9e6c058abe54168,sent,[texto removido - regras GLPD],sem quotes
396498,605f533e184ff8e0d52512f1,2021-03-27T15:46:06.787Z,5ff6cbd5d9e6c058abe54168,sent,[texto removido - regras GLPD],sem quotes
...,...,...,...,...,...,...
396555,61ecc0ce7c85e8f0fbffa0a7,2022-01-23T02:43:26.648Z,61ecbdfb8d82d2ff665ba3df,processed_pass,[texto removido - regras GLPD],sem quotes
396556,61ecc0dd7c85e8f0fbffa0a8,2022-01-23T02:43:41.502Z,61ecbdfb8d82d2ff665ba3df,processed_pass,[texto removido - regras GLPD],sem quotes
396557,61ecc16a7c85e8f0fbffa0a9,2022-01-23T02:46:02.240Z,61ecbdfb8d82d2ff665ba3df,processed_pass,[texto removido - regras GLPD],sem quotes
396558,61ed899c5f77401e3d11a329,2022-01-23T17:00:12.908Z,61ecbdfb8d82d2ff665ba3df,sent,[texto removido - regras GLPD],sem quotes
